In [19]:
!pip install strands-agents[mistral] python-dotenv

In [1]:
import json
import os
import sys
import dotenv
import boto3
import requests

import pprint

import yaml
from pathlib import Path
from typing import Dict, List, Optional, Tuple, TypeVar
from tqdm import tqdm

# Pydantic for structured data
from pydantic import BaseModel, Field

# Strands for AI agents
from strands.agent import Agent
from strands.models.mistral import MistralModel

# AWS authentication
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest

sys.path.append('..')

from src.utils import (
    save_json,
    read_json,
    load_file_content,
    get_job_paths,
    get_training_paths,
    sanity_check,
	chat_with_persona,
    track_api_call,  # Cost tracking from utils
    print_cost_summary,  # Cost summary from utils
    reset_cost_tracker  # Reset cost tracker from utils
)

from src.my_utils import (
    display_markdown_file,
    call_mistral,
    get_agent,
    batch_extract,
    send_message_to_chat
)

from src.models.persona_info import PersonaInfo

from src.models.interview_info import(
    InterviewInfo,
    InverviewQualityInfo,
    InterviewAgentMessage
)
from src.prompts.interview_prompt import(
    TRAINING_EXTENSION_INTERVIEW_PROMPT,
    TRAINING_EXTENSION_INTERVIEW_QUALITY_CHECK_PROMPT
)

# Load API key from .env file
dotenv.load_dotenv("../env")

# Check if we're good to go
if not os.getenv("MISTRAL_API_KEY"):
    print("❌ No MISTRAL_API_KEY found!")
    print("Create an env file with your API key")
else:
    print("✅ API key found, we're ready to roll")

✅ API key found, we're ready to roll


In [2]:
MAX_TURNS_IN_INTERVIEW = 10

In [3]:
DATA_PERSONAS_INFO_DIR = Path('../data_personas_info')
DATA_JOBS_DIR = Path('../data_jobs')
DATA_TRAININGS_DIR = Path('../data_trainings')
DATA_INTERVIEWS_DIR = Path('../data_interviews')

In [4]:
with open("../src/config.yaml", "r") as f:
    config = yaml.safe_load(f)

personas_info_data_version = config["personas_info_data_version"]
print(f"personas_info_data_version version: {personas_info_data_version}")

job_data_version = config["job_data_version"]
print(f"job_data_version version: {job_data_version}")

training_data_version = config["training_data_version"]
print(f"training_data_version version: {training_data_version}")

interview_data_version = config["interview_data_version"]
print(f"interview_data_version version: {interview_data_version}")

personas_info_data_version version: v14
job_data_version version: v4
training_data_version version: v7
interview_data_version version: v8


In [5]:
filename = f"map_clusters_jobs_{job_data_version}.json"
save_path = DATA_JOBS_DIR / filename
jobs_map = read_json(save_path)

In [6]:
# Load skills domains data
filename = f"final_map_clusters_trainings_{training_data_version}.json"
save_path = DATA_TRAININGS_DIR / filename
trainings_map = read_json(save_path)

print(f"✅ Loaded {len(trainings_map)} skills domains")
print("\n" + "="*50)

✅ Loaded 12 skills domains



In [7]:
# Load Personas data
filename = f"final_with_jobs_trainings_personas_info_{personas_info_data_version}.json"
personas_save_path = DATA_PERSONAS_INFO_DIR / filename

personas_data = read_json(personas_save_path)

# Convert to PersonaInfo objects
personas = {
    pid: PersonaInfo.model_validate_json(data)
    for pid, data in personas_data.items()
}

print(f"✅ Loaded {len(personas)} personas")
print("\n" + "="*50)

✅ Loaded 100 personas



# Perform interviews

In [8]:
def conduct_persona_interview(
    persona_id: str,
    domains_str: str,
    conversation_id: str = None,
    max_turns: int = 5,
    model: str = "mistral-medium-latest",
    print_conversation: bool = False
) -> List[str]:
    """Interview a persona and return conversation transcript"""
    
    interview = InterviewInfo()
    
    # prompt = INITIAL_INTERVIEW_PROMPT
    prompt = TRAINING_EXTENSION_INTERVIEW_PROMPT
    
    interview_agent = get_agent(prompt, model_id=model)

    # Start with greeting
    agent_message = "In a single sentence, which domains you are interested in to find trainings from following list:\n"
    agent_message += domains_str
    agent_message += "If none of them or not interested by a training, just say it"

    # print(conversation_id)
    # print(prompt)
    # print(agent_message)
    # return None
    
    if print_conversation:
        print(f"Assistant: {agent_message}")
    interview.interview.append(f"Assistant: {agent_message}")

    # Conduct interview
    for turn in range(max_turns):
        resp = send_message_to_chat(agent_message, persona_id, conversation_id)
        print(resp)

        if resp is None:
            break

        user_response, conversation_id = resp
        interview.conversation_id = conversation_id
        interview.interview.append(f"User: {user_response}")
        if print_conversation:
            print(f"User: {user_response}")

        # Generate next question
        conversation_str = '\n'.join(interview.interview)
        agent_response = interview_agent.structured_output(output_model=InterviewAgentMessage, prompt=conversation_str)
        # agent_response = interview_agent.structured_output(output_model=InterviewAgentMessage, user_response)

        # Track cost (using utils.py function)
        # track_api_call(agent_response, model)

        if agent_response.conversation_finished is True:
            break
            
        agent_message = agent_response.message
        interview.interview.append(f"Assistant: {agent_message}")
        if print_conversation:
            print(f"Assistant: {agent_message}")

    return interview

In [10]:
if False:
    persona_id = 'persona_001'

    conversation_id = None
    # print(conversation_id)
    # print(conversation)

    domains_str = ""
    for domain in trainings_map:
        domains_str += f"- {domain}" + "\n"

    print(domains_str)

    # interview = conduct_persona_interview(
    #     persona_id,
    #     domains_str,
    #     conversation_id=conversation_id,
    #     max_turns=5,
    #     model="mistral-small-latest",
    #     print_conversation=True
    # )

    # print(interview)

In [11]:
# Interview all personas
#persona_ids = [f'persona_{i:03}' for i in range(1, 4)]
persona_ids = [f'persona_{i:03}' for i in range(1, 101)]

# personas_save_path = SUBMISSION_DIR / 'personas.json'
filename = f"training_domain_extension_interviews_{interview_data_version}.json"
interviews_save_path = DATA_INTERVIEWS_DIR / filename
if not interviews_save_path.exists():
    save_json(interviews_save_path, {})
interviews = read_json(interviews_save_path)

# Track how many new personas we process
new_personas_processed = 0

for persona_id in tqdm(persona_ids):

    if persona_id in interviews:
        continue

    persona_info = personas[persona_id]
    if persona_info.recommendation_type != 'trainings_only':
        continue

    print(persona_id)
    conversation_id = None

    new_personas_processed += 1

    domains_str = ""
    for domain in trainings_map:
        domains_str += f"- {domain}" + "\n"

    # Interview
    conversation = conduct_persona_interview(
        persona_id,
        domains_str,
        conversation_id=conversation_id,
        max_turns=3,
        print_conversation=False)
    interviews[persona_id] = conversation.model_dump()

    # Save every interview
    if len(interviews) % 1 == 0:
        save_json(interviews_save_path, interviews)

    # Show cost update every 20 personas
    # if new_personas_processed > 0 and new_personas_processed % 20 == 0:
    #     print(f"\n💰 Cost update after {new_personas_processed} new personas:")
    #     print_cost_summary()
    #     print()

    if new_personas_processed > 0:break

save_json(interviews_save_path, interviews)

  0%|          | 0/100 [00:00<?, ?it/s]

persona_007
('The brush only listens to the whisper of colors.', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_007#2025-10-27T12:13:52')


  6%|▌         | 6/100 [00:05<01:20,  1.17it/s]


---

# For Debug Only

---

# Redo interview of Persona

In [15]:
if True:
    persona_id = 'persona_025'

    #filename = f"training_domain_extension_interviews_{interview_data_version}.json"
    #interviews_save_path = DATA_INTERVIEWS_DIR / filename
    #interviews = read_json(interviews_save_path)
    #conversation_id = None

    filename = f"full_interviews_{persona_id}.json"
    interview_save_path = DATA_INTERVIEWS_DIR / filename
    interview = read_json(interview_save_path)
    conversation_id = interview['conversation_id']
    
    domains_str = ""
    for domain in trainings_map:
        domains_str += f"- {domain}" + "\n"

    # Interview
    conversation = conduct_persona_interview(
        persona_id,
        domains_str,
        conversation_id=conversation_id,
        max_turns=5,
        print_conversation=True)

    new_interview = conversation.model_dump()
    interview['interview'].extend(new_interview['interview'])
    save_json(interview_save_path, interview)

    #interviews[persona_id] = conversation.model_dump()
    
    #print(conversation)
    #save_json(interviews_save_path, interviews)

Assistant: In a single sentence, which domains you are interested in to find trainings from following list:
- Financial Risk Management And Compliance
- Electrical And Electronic Systems Engineering
- Food Safety And Management
- Fiber And Paper Industry Operations
- Industrial Equipment Maintenance And Optimization
- Procurement And Supply Chain Management
- Hospitality And Tourism Management
- Legal Practice And Advocacy
- Maritime And Port Operations Management
- Visual And Artistic Skills
- Information Management And Digital Security
- Live Event Technical Management
If none of them or not interested by a training, just say it
('The one about Industrial Equipment Maintenance and Optimization sounds cool because of the machines!', '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_025#2025-10-30T20:26:15')
User: The one about Industrial Equipment Maintenance and Optimization sounds cool because of the machines!


/tmp/ipykernel_13828/3774725164.py:48: DeprecationWarning: Agent.structured_output method is deprecated. You should pass in `structured_output_model` directly into the agent invocation. see: https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/structured-output/
  agent_response = interview_agent.structured_output(output_model=InterviewAgentMessage, prompt=conversation_str)


# Check interview qualities

In [8]:
def check_interview_quality(
    interview: str,
    model: str = "mistral-small-latest",
    print_prompt: bool = False
) -> InverviewQualityInfo:
    prompt = TRAINING_EXTENSION_INTERVIEW_QUALITY_CHECK_PROMPT.format(
        interview=interview
    )

    if print_prompt is True:
        print(prompt)
    
    extraction_agent = get_agent(model_id=model, temperature=0.0)
    result = extraction_agent.structured_output(output_model=InverviewQualityInfo, prompt=prompt)

    return result

In [9]:
filename = f"training_domain_extension_interviews_{interview_data_version}.json"
interviews_save_path = DATA_INTERVIEWS_DIR / filename
interviews = read_json(interviews_save_path)

In [10]:
filename = f"final_with_jobs_trainings_personas_info_{personas_info_data_version}.json"
personas_save_path = DATA_PERSONAS_INFO_DIR / filename
personas_data = read_json(personas_save_path)

In [11]:
cache_period = 5

filename = f"quality_training_domain_extension_interviews__{interview_data_version}.json"
save_path = DATA_INTERVIEWS_DIR / filename
if not save_path.exists():
    save_json(save_path, {})
quality_interviews = read_json(save_path)

new_items_processed = 0
for persona_id in tqdm(personas_data):
    persona_data_dict = json.loads(personas_data[persona_id])

    if persona_data_dict['recommendation_type'] != 'trainings_only':
        continue

    if persona_id in quality_interviews:
        quality = json.loads(quality_interviews[persona_id])
        if quality['quality_level'] == 'OK':
            continue

    if persona_id not in interviews:
        quality_data = {
            'quality_level': 'NOK',
            'rationale': 'interview missing'
        }
        quality = InverviewQualityInfo(**quality_data)
        quality_str = json.dumps(quality.model_dump(), ensure_ascii=False)
        quality_interviews[persona_id] = quality_str
        save_json(save_path, quality_interviews)
        continue

    new_items_processed = new_items_processed + 1

    interview = interviews[persona_id]['interview']

    interview_str = "\n".join(interview)
    # print(interview_str)

    quality = check_interview_quality(
        interview_str,
        print_prompt=False
    )
    quality_str = json.dumps(quality.model_dump(), ensure_ascii=False)

    quality_interviews[persona_id] = quality_str

    if new_items_processed % cache_period == 0:
        save_json(save_path, quality_interviews)

    #if new_items_processed > 0:break

save_json(save_path, quality_interviews)

  0%|          | 0/100 [00:00<?, ?it/s]

{'conversation_id': '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_046#2025-10-23T07:09:51', 'interview': ['Assistant: In a single sentence, which domains you are interested in to find trainings from following list:\n- Financial Risk Management And Compliance\n- Electrical And Electronic Systems Engineering\n- Food Safety And Management\n- Fiber And Paper Industry Operations\n- Industrial Equipment Maintenance And Optimization\n- Procurement And Supply Chain Management\n- Hospitality And Tourism Management\n- Legal Practice And Advocacy\n- Maritime And Port Operations Management\n- Visual And Artistic Skills\n- Information Management And Digital Security\n- Live Event Technical Management\nIf none of them or not interested by a training, just say it', 'User: I think Legal Practice and Advocacy… but maybe not the others, I’m not sure…']}


 46%|████▌     | 46/100 [00:00<00:00, 59.80it/s]

{'conversation_id': '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_053#2025-10-23T07:09:57', 'interview': ['Assistant: In a single sentence, which domains you are interested in to find trainings from following list:\n- Financial Risk Management And Compliance\n- Electrical And Electronic Systems Engineering\n- Food Safety And Management\n- Fiber And Paper Industry Operations\n- Industrial Equipment Maintenance And Optimization\n- Procurement And Supply Chain Management\n- Hospitality And Tourism Management\n- Legal Practice And Advocacy\n- Maritime And Port Operations Management\n- Visual And Artistic Skills\n- Information Management And Digital Security\n- Live Event Technical Management\nIf none of them or not interested by a training, just say it', "User: I think I'd be interested in Financial Risk Management And Compliance, but I'm not sure if it's too far from accounting and data stuff."]}


 53%|█████▎    | 53/100 [00:01<00:01, 36.76it/s]

...

 59%|█████▉    | 59/100 [00:01<00:01, 25.38it/s]

{'conversation_id': '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_067#2025-10-23T19:50:09', 'interview': ['Assistant: In a single sentence, which domains you are interested in to find trainings from following list:\n- Financial Risk Management And Compliance\n- Electrical And Electronic Systems Engineering\n- Food Safety And Management\n- Fiber And Paper Industry Operations\n- Industrial Equipment Maintenance And Optimization\n- Procurement And Supply Chain Management\n- Hospitality And Tourism Management\n- Legal Practice And Advocacy\n- Maritime And Port Operations Management\n- Visual And Artistic Skills\n- Information Management And Digital Security\n- Live Event Technical Management\nIf none of them or not interested by a training, just say it', 'User: Honestly, the only one that kinda catches my eye is Financial Risk Management And Compliance, but I’m really just looking for something focused on Policy Implementation in banking or insurance.']}


 67%|██████▋   | 67/100 [00:02<00:01, 20.55it/s]

{'conversation_id': '39eba84b-46b0-4bc7-b29c-ceae08d4d602#persona_092#2025-10-23T07:10:08', 'interview': ['Assistant: In a single sentence, which domains you are interested in to find trainings from following list:\n- Financial Risk Management And Compliance\n- Electrical And Electronic Systems Engineering\n- Food Safety And Management\n- Fiber And Paper Industry Operations\n- Industrial Equipment Maintenance And Optimization\n- Procurement And Supply Chain Management\n- Hospitality And Tourism Management\n- Legal Practice And Advocacy\n- Maritime And Port Operations Management\n- Visual And Artistic Skills\n- Information Management And Digital Security\n- Live Event Technical Management\nIf none of them or not interested by a training, just say it', 'User: Only Culture and Document Management, but if I gotta pick from that list, then Information Management and Digital Security—it’s the closest to what I need for digital archiving and data security trainings.']}


100%|██████████| 100/100 [00:03<00:00, 26.62it/s]


In [12]:
filename = f"quality_training_domain_extension_interviews__{interview_data_version}.json"
save_path = DATA_INTERVIEWS_DIR / filename
quality_interviews = read_json(save_path)

for persona_id in quality_interviews:
    quality = json.loads(quality_interviews[persona_id])
    if quality['quality_level'] != 'OK':
        print(persona_id)
        print(quality['rationale'])
        print('---')

persona_059
The user did not provide information about any of the listed training domains. The user expressed interest in a domain not listed in the provided options.
---


# Redo interview of a Persona

In [9]:
persona_id = "persona_009"

# Interview
print("🎤 Conduct Interview...")
conversation = conduct_persona_interview(persona_id, max_turns=10, print_conversation=True)
print(conversation)

🎤 Conduct Interview...


TypeError: conduct_persona_interview() missing 1 required positional argument: 'domains_str'

In [11]:
filename = f"job_extension_interviews_{interview_data_version}.json"
interviews_save_path = DATA_INTERVIEWS_DIR / filename
interviews = read_json(interviews_save_path)
interviews[persona_id] = conversation.model_dump()
save_json(interviews_save_path, interviews)

NameError: name 'conversation' is not defined

# Translate interviews in english

In [ ]:
def translate_interview(
    interview,
    model: str = "mistral-small-latest",
    print_prompt=False
) -> InterviewInfo:

    prompt = TRANSLATE_INTERVIEW_PROMPT.format(
        interview=interview
    )

    if print_prompt is True:
        print(prompt)

    extraction_agent = get_agent(model_id=model, temperature=0.0)
    result = extraction_agent.structured_output(output_model=InterviewInfo, prompt=prompt)

    return result

In [ ]:
filename = f"interviews_{interview_data_version}.json"
interviews_save_path = DATA_INTERVIEWS_DIR / filename
interviews = read_json(interviews_save_path)

In [ ]:
cache_period = 5

filename = f"en_interviews_{interview_data_version}.json"
save_path = DATA_INTERVIEWS_DIR / filename
if not save_path.exists():
    save_json(save_path, {})
en_interviews = read_json(save_path)


new_items_processed = 0
for interview_id in tqdm(interviews):
    if interview_id in en_interviews:
        continue

    new_items_processed = new_items_processed + 1
    
    interview = interviews[interview_id]

    translated_interview = translate_interview(interview, print_prompt=False)
    en_interviews[interview_id] = translated_interview.interview

    if new_items_processed % cache_period == 0:
        save_json(save_path, en_interviews)

    # if new_items_processed > 4:break

save_json(save_path, en_interviews)